In [56]:


def format_insert(tname, thedic):
    for i,j in thedic.items(): 
        if isinstance(j, list): 
            thedic[i]= f"ARRAY {j}"
    keystr = ",".join([f"{i}" for i,j in thedic.items()]); 
    valstr = ",".join([f"{j}" for i,j in thedic.items()]); 
    
    finalstr = f"""INSERT INTO {tname} ({keystr}) VALUES ({valstr});"""
    return finalstr

def read_data(result): 
    retdata = pd.DataFrame(result, columns=["f_id", "feature", "point_cloud", "density", "gen_date"])
    return retdata

def point_density(feature):
    feature = str(feature); 
    density = len(feature.replace("0", ""))/len(feature)
    return density

x = np.random.normal(20, 1.5, 100)
y = np.random.normal(10, 8.6, 100)
z = np.random.normal(35, 5.4, 100)

point_stack = np.vstack((x,y,z)).T.tolist()
feature_64 = hash().__str__()[:4]*16
thedic = {
    "feature" : feature_64, 
    "density" : point_density(feature_64), 
    "point_cloud": point_stack, 
}

teststr = format_insert("mass", thedic)



conn = pg.connect("dbname='testfeatures' user='yang' host='localhost' password='testyang'")

cur = conn.cursor()

cur.execute(teststr)
cur.execute("SELECT * from mass;")

results = cur.fetchall()
print(results)

conn.commit()
cur.close()
conn.close()


[(2, '4', [[Decimal('1.0000'), Decimal('2.0000'), Decimal('3.0000')], [Decimal('2.0000'), Decimal('3.0000'), Decimal('4.0000')]], Decimal('0.230'), None), (1, '5', [[Decimal('1.0000'), Decimal('2.0000'), Decimal('3.0000')], [Decimal('2.0000'), Decimal('3.0000'), Decimal('4.0000')]], Decimal('0.230'), None), (3, '6', [[Decimal('1.0000'), Decimal('2.0000'), Decimal('3.0000')], [Decimal('2.0000'), Decimal('3.0000'), Decimal('4.0000')]], Decimal('0.230'), datetime.datetime(2023, 2, 4, 8, 41, 8, 392468)), (4, '6', [[Decimal('1.0000'), Decimal('2.0000'), Decimal('3.0000')], [Decimal('2.0000'), Decimal('3.0000'), Decimal('4.0000')]], Decimal('0.230'), datetime.datetime(2023, 2, 4, 8, 41, 19, 282975)), (5, '6', [[Decimal('1.0000'), Decimal('2.0000'), Decimal('3.0000')], [Decimal('2.0000'), Decimal('3.0000'), Decimal('4.0000')]], Decimal('0.230'), datetime.datetime(2023, 2, 4, 8, 47, 38, 93590)), (6, '6', [[Decimal('18.1900'), Decimal('6.9300'), Decimal('40.1400')], [Decimal('16.6300'), Decimal

### Data Structure 
<pre>
 f_id        | integer               |           | not null | nextval('mass_f_id_seq'::regclass)
 feature     | character varying(64) |           |          | 
 point_cloud | numeric[]             |           |          | 
 density     | numeric(4,3)          |           |          | 
 gen_date    | date
</pre>

In [134]:
import json, time
import psycopg2 as pg
import pandas as pd 
import numpy as np 

class Connection:
    def __init__(self):
        self.connect = False; 
        self.__curs = []; 
        self.__connect_info = {
            "user": "user", 
            "dbname": "database", 
            "host": "localhost", 
            "password": "",
        }
        
    def __insert_str_dict(self, tname, thedic):
        for i,j in thedic.items(): 
            if isinstance(j, list): 
                thedic[i]= f"ARRAY {j}"
        keystr = ",".join([f"{i}" for i,j in thedic.items()]); 
        valstr = ",".join([f"{j}" for i,j in thedic.items()]); 
        finalstr = f"""INSERT INTO {tname} ({keystr}) VALUES ({valstr});"""
        return finalstr
    
    def __connect_str(self): 
        strlst = [f"{i}='{j}'" for i,j in self.__connect_info.items() if len(j) > 0]; 
        finalstr = " ".join(strlst); 
        return finalstr; 
    
    def read_conf(self, conffile): 
        with open(conffile) as file1: 
            confs = json.load(file1); 
        for i in self.__connect_info.keys(): 
            if i in confs.keys(): 
                self.__connect_info[i] = confs[i]; 
    
    def connectit(self):
        connectstr =self.__connect_str(); 
        try: 
            print(f"Successfully connect to the database", self.__connect_info["dbname"], "as user", self.__connect_info["user"])
            self.connect = pg.connect(connectstr); 
        except: 
            print("Fatal: Cannot connect to the database based on the given info");
            print(f"Please carefully check the following connection setting: \n{connectstr}"); 
        
    def insert(self, tname, thedic, clean=True): 
        insertstr = self.__insert_str_dict(tname, thedic); 
        cur = self.connect.cursor(); 
        try: 
            cur.execute(insertstr);
            self.connect.commit(); 
            if clean != True: 
                self.__curs.append(cur); 
            else: 
                cur.close(); 
        except: 
            print("Failed to execute the insertion. "); 
            cur.close(); 
            
    def doit(self, cmd): 
        cur = self.connect.cursor(); 
        try: 
            cur.execute(cmd); 
            self.connect.commit(); 
            cur.close(); 
        except: 
            print(f"Failed to execute the command. {cmd}"); 
            cur.close(); 
        return results
    
    def select(self, selectstr, clean=True): 
        cur = self.connect.cursor(); 
        try: 
            cur.execute(selectstr); 
            results = cur.fetchall(); 
            if clean != True: 
                self.__curs.append(cur); 
            else: 
                cur.close(); 
        except: 
            print("Failed to execute the selection. "); 
            cur.close(); 
        return results
    
    def clear(self):
        self.connect.close()
        if len(self.__curs) > 0:
            print("cleaning cursors")
            [i.close() for i in self.__curs]; 
        
    
# cur.execute("SELECT * from mass;"); 

conffile = "/Users/yangzhang/Dropbox/Documents/BetaPose/myconfig.json"
agent = Connection();
agent.read_conf(conffile); 
agent.connectit(); 

domodify = False

if domodify: 
    print("doing insertion")
    point_stack = np.vstack((x,y,z)).T.tolist()
    feature_64 = time.perf_counter().__str__()[-8:]*8
    thedic = {
        "feature" : feature_64, 
        "density" : point_density(feature_64), 
        "point_cloud": point_stack, 
    }
    
    agent.update("mass", thedic)
agent.doit("delete from mass where f_id<8")
ret = agent.select("SELECT * from mass;"); 
ret2 = agent.select("SELECT * from mass where density <=0.4;"); 
    
agent.clear()
# print(len(ret2))
# print(ret2)

Successfully connect to the database testfeatures as user yang


In [38]:

x = np.random.normal(20, 1.5, 100)
y = np.random.normal(10, 8.6, 100)
z = np.random.normal(35, 5.4, 100)

thestack = np.vstack((x,y,z)).T
print(thestack.shape)
print()
# print(thestack.T)
thestack.tolist()


(100, 3)



[[18.45444927415225, 11.83706291229125, 34.153550793133476],
 [18.927857402707858, 13.098999451919, 31.926595566727755],
 [19.816363065001617, 1.6675247997582474, 30.871189319264072],
 [19.006546580655424, 10.00785099692746, 32.206521429673224],
 [20.959810235870457, 11.78925111389129, 31.75340848847469],
 [18.619404130419504, 13.425251512194686, 26.782755649892934],
 [21.024324464362248, 8.584698272540276, 32.4104586653563],
 [19.591727539703527, 16.233460477422447, 35.23943091344019],
 [19.85000701771609, 17.262345433045496, 38.26346337059736],
 [20.08561703964838, 18.518691273818558, 28.84870567116966],
 [19.084204001340535, 8.240140861633312, 39.12501882700427],
 [18.24856021470269, 13.988220109577851, 40.85944027793226],
 [20.532957485574137, -8.666508900451309, 39.683751662142896],
 [19.930241413441447, 9.62032971088483, 36.64114867410529],
 [19.677948356383997, 6.67704628987126, 37.731748142719624],
 [20.667423142324246, 13.557237778611908, 29.86187105009686],
 [19.1872727851317

In [135]:
read_data(ret)

,f_id,feature,point_cloud,density,gen_date
0,8,6666666666666666666666666666666666666666666666...,"[[19.5800, 23.5900, 31.0400], [19.4800, -14.73...",0.230,2023-02-04 15:04:39.470225
1,9,6666666666666666666666666666666666666666666666...,"[[21.6000, 12.8800, 27.8500], [19.4200, 9.8300...",0.230,2023-02-04 15:07:25.885993
2,10,6666666666666666666666666666666666666666666666...,"[[18.4400, 7.0800, 32.8900], [22.5900, 9.0200,...",0.230,2023-02-04 15:08:13.223228
3,11,6666666666666666666666666666666666666666666666...,"[[15.7642, -1.9989, 37.6551], [19.0877, 6.8977...",0.230,2023-02-04 15:08:30.266822
4,12,1010101010101010101010101010101010101010101010...,"[[17.8709, 9.8465, 37.1919], [17.3832, 8.4380,...",0.230,2023-02-04 15:09:57.189455
5,13,1010101010101010101010101010101010101010101010...,"[[16.8857, 6.4920, 31.1023], [21.4408, 8.0397,...",0.500,2023-02-04 15:16:10.927599
6,25,2635566626355666263556662635566626355666263556...,"[[16.8857, 6.4920, 31.1023], [21.4408, 8.0397,...",1.000,2023-02-04 16:58:42.201948
7,26,1338579113385791133857911338579113385791133857...,"[[16.8857, 6.4920, 31.1023], [21.4408, 8.0397,...",1.000,2023-02-04 16:58:53.189010


In [74]:
import json

connect_info = {
    "user": "user", 
    "dbname": "database", 
    "host": "localhost", 
}
print(json.dumps(connect_info, indent=2     ))

{
  "user": "user",
  "dbname": "database",
  "host": "localhost"
}


In [1]:
import pytraj as pt 